In [2]:
import os
from bs4 import BeautifulSoup
from natasha import NamesExtractor  
import json
extractor = NamesExtractor()
import re
#регулярка для имён
fio = re.compile("[А-ЯЁ][а-яё]+ [А-ЯЁ]\.[А-ЯЁ]\.")
fio_cifer = re.compile("[А-ЯЁ][а-яё]+ ФИО[0-9]{1,3}")
fio_short = re.compile("ФИО[0-9]{1,3}")
import pandas as pd
import random

In [3]:
from meta_extraction import final_meta, meta_lines, get_meta_names
from meta_extraction import get_parts, parse_xml

In [4]:
import numpy as np

### Сбор данных для оценки. 

In [5]:
def getting_paths(path_to_folder):
    paths = []
    for f in os.walk(path_to_folder):
        for path in f[2]:
            paths.append(path_to_folder+"/"+path)
    return paths

In [5]:
def only_criminal(paths):
    new_paths = []
    for path in paths:
        text = ""
        with open(path, "r") as f:
            for line in f.readlines():
                text += line
        if "<vidpr>Уголовное</vidpr>" not in text: 
            os.remove(path)
        else:
            new_paths.append(path) 
    return new_paths            

In [6]:
persons_meta = ["prosecutor", "advocate", "accused", "secretary"]

In [8]:
def random_corpus(link, number):
    links_to_folders = []
    links_to_files = []
    for x in os.walk(link):
        links_to_folders.append(x[0])
    for link in links_to_folders:
        links_to_files += getting_paths(link)
    all_links_to_files = [link for link in links_to_files if "//" not in link]
    corpus_link = random.sample(all_links_to_files, number)
    return corpus_link

In [9]:
rand_links = random_corpus("/home/laidhimonthegreen/Документы/coursework/evaluation_data/", 100) 

Ссылки были сохранены в файл links_standard.txt

In [1]:
our_links = []
with open("links_standard.txt", "r") as f:
    for line in f.readlines():
        our_links.append(line.strip())

Золотой стандарт был предварительно размечен вручную. 

In [10]:
goldstandard = pd.DataFrame.from_csv("/home/laidhimonthegreen/Документы/coursework/gold_standard.csv", sep = "\t")

/home/laidhimonthegreen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [9]:
roles = ["number","prosecutor", "accused", "secretary", "advocate"]

In [10]:
script_martkup_df = pd.DataFrame(columns = roles)

Добавляем данные, извлеченные из корпуса автоматически. 

In [8]:
prosecutors = []
accuseds = []
secretaries = []
advocates = []

In [9]:
for i, link in enumerate(our_links): 
    try: 
        roledict = final_meta(link)    
        accuseds.append(roledict["accused"])
        prosecutors.append(roledict["prosecutor"])
        secretaries.append(roledict["secretary"])
        advocates.append(roledict["advocate"])
    except IndexError:
        prosecutors.append("not found")
        accuseds.append("not found")
        secretaries.append("not found")
        advocates.append("not found")

### Оценка.

correct -- совпадает полностью  
partial -- ошибка в падежах, пересекается ФИО, совпадает один из людей  
incorrect -- полная ошибка  
spurious -- not found, но что-то нашли  
missing -- что-то есть в золотом стандарте, но not found  

In [25]:
#функция, которая ищет частичные совпадения 
def get_partials(name1, name2):
    a = len(name1.split()[0][0:-2])
    if len(name2) > a and " " in name1 and " " in name2:
        if name1.split()[0][:a] + " " + name1.split()[1] == name2.split()[0][:a] + " " + name2.split()[1]: 
            return 1
        else:
            return 0
    else:
        return 0

In [26]:
#достаём нужные нам переменные
def get_variables(preds, standard):
    not_cases, correct, partial, missing, spurious, incorrect = 0, 0, 0, 0, 0, 0
    for i, name in enumerate(preds):
        name1, name2 = preds[i], standard[i] 
        if name1 == "not found" and name2 == "not found":
            not_cases += 1
        elif name1 == "not found":
            missing += 1
        elif name2 == "not found":
            spurious += 1
        elif name1 == name2:
            correct += 1 
        elif get_partials(name1, name2) == 1: 
            partial += 1
        elif "," in name1:
            names1 = name1.split(",")
            names2 = name2.split(",")
            if get_partials(names1[0], names2[0]) or get_partials(names1[1], names2[0]) or get_partials(names1[0], names2[-1]) or get_partials(names1[1], names2[-1]):
                partial += 1 
        else:
            incorrect += 1
    return not_cases, correct, partial, missing, spurious, incorrect

In [27]:
#метрика
def get_metric(variables):
    correct = variables[1]
    partial = variables[2]
    incorrect = variables[5]
    missing = variables[3]
    spurious = variables[4]
    
    numerator = correct + 0.5*partial
    precision = round(numerator/(correct + spurious + partial + incorrect), 2)
    recall = round(numerator/(correct + missing + partial + incorrect), 2)
    
    return precision, recall

In [28]:
secretaries_gold = list(goldstandard["secretary"])
accuseds_gold = list(goldstandard["accused"])
prosecutors_gold = list(goldstandard["prosecutor"])
advocates_gold = list(goldstandard["advocate"])
all_people = advocates + secretaries + accuseds + prosecutors
all_people_gold = advocates_gold + secretaries_gold + accuseds_gold + prosecutors_gold

In [29]:
print (get_metric(get_variables(prosecutors, prosecutors_gold)),
        get_metric(get_variables(advocates, advocates_gold)), 
        get_metric(get_variables(secretaries, secretaries_gold)),
        get_metric(get_variables(accuseds, accuseds_gold)))

(0.79, 0.8) (0.84, 0.84) (0.78, 0.78) (0.7, 0.64)


In [30]:
get_metric (get_variables(all_people, all_people_gold))

(0.78, 0.76)